In [ ]:
proj_id = 'cmat-315920'
root_path = '/home/jupyter'
code_path = root_path + '/MathVGerrmandering_CMAT_2021/src/'
for f in ['Core', 'Gerry', 'Variable', 'Crosswalks', 'Assignments', 'Shapes',
          'Census', 'Elections', 'Votes', 'Combined', 'Edges', 'Nodes', 'Graph', 'Districts']:
    %run -i {code_path+f+'.py'}

refresh_all = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

refresh_tbl = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

self = Gerry(abbr = 'TX',
             level = 'cntyvtd',
             district_type='cd',
             simplification=300,
             num_colors=10,
             election_filters=(
                 "office='President' and race='general'",
                 "office='USSen' and race='general'",
#                  "office like 'USRep%' and race='general'",
             ),
             refresh_tbl=refresh_tbl, refresh_all=refresh_all
            )

start = time.time()

# idx = self.nodes.df.nlargest(2, 'total_pop').index
# self.nodes.df.loc[idx[0], 'cd'] = '37'
# self.nodes.df.loc[idx[1], 'cd'] = '38'

self.MCMC(num_steps=1)
self.agg_plans(agg_polygon_steps=True)#agg_polygon_steps=list(range(3,15)))
self.stack_plans()

elapsed = time.time() - start
h, m = divmod(elapsed, 3600)
m, s = divmod(m, 60)
print(f'{int(h)}hrs {int(m)}min {s:.2f}sec elapsed')

random_seed (default=1) 


Get crosswalks_TX_2020_tabblock   ... not necessary ... success
Get assignments_TX_2020_tabblock  ... tabblock table exists ... success
Get shapes_TX_2020_tabblock       ... tabblock table exists ... success
Get census_TX_2020_tabblock       ... tabblock table exists ... success
Get elections_TX_2020_tabblock    ... tabblock table exists ... success
Get votes_all_TX_2020_tabblock    ... tabblock table exists ... success
Get votes_hl_TX_2020_tabblock     ... tabblock table exists ... success
Get combined_TX_2020_cntyvtd_cd   ... cntyvtd table exists ... success
Get nodes_TX_2020_cntyvtd         ... success
Get edges_TX_2020_cntyvtd         ... cntyvtd table exists ... success
Get graph TX 2020 cntyvtd cd      ... gpickle exists ... success
Get cd TX 2020 cntyvtd cd         ... success
MCMC plan_1 ... Current pop imbalance = 32.71% ... setting tol = 32.71% ... found recomb with cd 29 and 22 split with pop_imbalance=29.11% ... 2763042801260912699 ... success
MCMC done


In [21]:
query = f"""
select
    geoid,
    case when A=B then cnty else cntyvtd end as cntyvtd
from (
    select
        geoid,
        cnty,
        cntyvtd,
        max({self.district_type}) over (partition by cnty) as A,
        min({self.district_type}) over (partition by cnty) as B
    from
        {self.assignments.tbl}
    )
"""

df = run_query(query)
df


# case when var(cast({self.district_type} as float))=0 then cnty else cntyvtd over (partition by cnty) as cntyvtd

,geoid,cntyvtd
0,481899501001000,48189
1,481899501001001,48189
2,481899501001002,48189
3,481899501001003,48189
4,481899501001004,48189
...,...,...
668752,484391229022022,439002569
668753,484391229022023,439002569
668754,484391229022024,439002569
668755,484391229022025,439002569


In [22]:
self.tbl

'cmat-315920.redistricting_data.cook_plans_TX_2020_cntyvtd_cd'